subset of categories: cougar_face, helicopter, pizza, helicopter

- select a set of training images
- extract SIFT features from training images (ignore position, orientation and scale)
- Concatenate features into matrix, one descriptor per row